# DES Simulation

This notebook generates the results used in the report for the second assignment of the course 'Stochastic Simulation'.

Authors:

1. Divyaben Hasmukhbhai Gajera
- Affiliation: MSc Computational Science, University of Amsterdam
- Email: divya.gajera@student.uva.nl
- Student Number: 14932644

2. K. López
- Affiliation: MSc Computational Science, University of Amsterdam
- Email: kenia.lopez.sotomayor@student.uva.nl
- Student Number: 12965081

3. T. P. Glansdorp
- Affiliation: MSc Computational Science, University of Amsterdam
- Email: thomas.glansdorp@student.uva.nl
- Student Number: 12748587

In [34]:
import simpy as sp
import numpy as np
import matplotlib.pyplot as plt
import random
import queue

## DES program that implements FIFO scheduling case

In [30]:
import simpy
import queue

def arrive(env, arrival_rate, queue):
    while True:
        inter_arrival_time = random.expovariate(arrival_rate)
        yield env.timeout(inter_arrival_time)
        queue.append(env.now)
        print(f"New arrival at: {env.now}")

def depart(env, service_rate, q, n):
    time_empty_server = env.now
    while True:
        yield env.timeout(random.expovariate(service_rate))
        if not len(q) == 0:
            arrival_time = q[0]
            wait_time = env.now - arrival_time
            print(f"Service at time: {env.now} and empty server at: {env.now + wait_time}")
            yield env.timeout(env.now + wait_time)
            print(env.timeout(env.now + wait_time))
            print(q)
            q.pop(0)
            print(q)

def MMn_queue(arrival_rate, service_rate, simulation_time, n):
    q = []
    env = simpy.Environment()

    env.process(arrive(env, arrival_rate, q))
    env.process(depart(env, service_rate, q, n))

    env.run(until=simulation_time)

arrival_rate = 2.0
service_rate = 3.0
n = 1
simulation_time = 10


MMn_queue(arrival_rate, service_rate, simulation_time, n)




New arrival at: 1.2578270096886255
Service at time: 1.515348842498136 and empty server at: 1.7728706753076466
New arrival at: 2.372899581601592
New arrival at: 2.7154236586860714
<Timeout(3.5457413506152937) object at 0x20096ffeef0>
[1.2578270096886255, 2.372899581601592, 2.7154236586860714]
[2.372899581601592, 2.7154236586860714]
New arrival at: 3.6247772950116106
Service at time: 3.7921551876266903 and empty server at: 5.211410793651789
New arrival at: 4.9926426261458134
New arrival at: 5.031909444451779
New arrival at: 5.057094937468837
New arrival at: 5.098534927742391
New arrival at: 5.181133197044442
New arrival at: 5.766078887894832
New arrival at: 5.7868585047158065
New arrival at: 6.93979562210673
New arrival at: 7.859076938472194
<Timeout(10.422821587303577) object at 0x20096ffee30>
[2.372899581601592, 2.7154236586860714, 3.6247772950116106, 4.9926426261458134, 5.031909444451779, 5.057094937468837, 5.098534927742391, 5.181133197044442, 5.766078887894832, 5.7868585047158065, 6

In [66]:
class MMnFIFOQueue:
    def __init__(self, env, arrival_rate, service_rate, n):
        self.env = env
        self.server = simpy.Resource(env, capacity=n)
        self.arrival_rate = arrival_rate
        self.service_rate = service_rate
        self.queue = queue.Queue()

        env.process(self.arrive())
        

    def arrive(self):
        while True:
            inter_arrival_time = random.expovariate(self.arrival_rate)
            yield self.env.timeout(inter_arrival_time)
            self.env.process(self.depart())
            
    def depart(self):
        with self.server.request() as request:
            yield request
            service_time = random.expovariate(self.service_rate)
            yield self.env.timeout(service_time)


def run_MMn_FIFO_queue(arrival_rate, service_rate, n, simulation_time):
    env = simpy.Environment()
    mmn_queue = MMnFIFOQueue(env, arrival_rate, service_rate, n)
    env.run(until=simulation_time)


In [67]:
arrival_rate = 2.0
service_rate = 3.0
simulation_time = 10.0
n = 1

run_MMn_FIFO_queue(arrival_rate, service_rate, n, simulation_time)

### Statistical significance 

### Numer of measurements required

## DES program that implements shortest job first case for M/M/1

In [64]:
class MMnSJFQueue:
    def __init__(self, env, arrival_rate, service_rate, n):
        self.env = env
        self.server = simpy.Resource(env, capacity=n)
        self.arrival_rate = arrival_rate
        self.service_rate = service_rate
        self.queue = simpy.PriorityStore(env)

        env.process(self.arrive())
        

    def arrive(self):
        while True:
            inter_arrival_time = random.expovariate(self.arrival_rate)
            yield self.env.timeout(inter_arrival_time)
            self.queue.put(self.env.now)
            self.env.process(self.depart())
            
    def depart(self):
        with self.server.request() as request:
            yield request
            service_time = random.expovariate(self.service_rate)
            yield self.env.timeout(service_time)


def run_MMn_SJF_queue(arrival_rate, service_rate, n, simulation_time):
    env = simpy.Environment()
    mmn_queue = MMnSJFQueue(env, arrival_rate, service_rate, n)
    env.run(until=simulation_time)

In [65]:
arrival_rate = 2.0
service_rate = 3.0
simulation_time = 10.0
n = 1

run_MMn_SJF_queue(arrival_rate, service_rate, n, simulation_time)

## Different service rate distribution on M/D/n and M/D/1

## Long-tail distribution